<a href="https://colab.research.google.com/github/naguzmans/opportunistic-atm/blob/master/04_ConvLSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>